In [ ]:
import pandas as pd

In [ ]:
file_paths = [
    'verified-smart-contracts/test/part.0.parquet',
    'verified-smart-contracts/validation/part.0.parquet',
    'verified-smart-contracts/train/part.0.parquet',
    'verified-smart-contracts/train/part.1.parquet',
    'verified-smart-contracts/train/part.2.parquet',
    'verified-smart-contracts/train/part.3.parquet',
    'verified-smart-contracts/train/part.4.parquet',
]

df = pd.concat([pd.read_parquet(file) for file in file_paths], ignore_index=True)
df.info()

In [ ]:
results = df[df['source_code'].str.contains("block.number|block.timestamp")]

In [ ]:
results = results.drop_duplicates(subset='contract_address', keep='first')

In [ ]:
results.reset_index(drop=True, inplace=True)